# Exercises

In this section we have two exercises:
1. Implement the polynomial kernel.
2. Implement the multiclass C-SVM.

## Polynomial kernel

You need to extend the ``build_kernel`` function and implement the polynomial kernel if the ``kernel_type`` is set to 'poly'. The equation that needs to be implemented:
\begin{equation}
K=(X^{T}*Y)^{d}.
\end{equation}

In [257]:
# Import necessary libraries
import numpy as np
import cvxopt
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

cvxopt.solvers.options['show_progress'] = True

In [258]:
def build_kernel(data_set,
                 kernel_type='linear',
                 gamma=1.0,
                 coef0=1.0,
                 degree=2):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        n = len(data_set)
        ones = np.ones((n, 1))
        kernel -= 0.5 * (np.dot(np.diag(kernel).reshape(-1, 1), ones.T) +
                         np.dot(ones, np.diag(kernel).reshape(1, -1)))
        kernel = np.exp(kernel / (2.0 * sigma ** 2))
    elif kernel_type == 'poly':
        kernel = (gamma * kernel + coef0) ** degree
    return kernel

## Implement a multiclass C-SVM

Use the classification method that we used in notebook 7.3 and IRIS dataset to build a multiclass C-SVM classifier. Most implementation is about a function that will return the proper data set that need to be used for the prediction. You need to implement:
- ``choose_set_for_label``
- ``get_labels_count``

In [259]:
def choose_set_for_label(data_set, labels, classes,
                         train_ratio=0.7, random_state=None):
    idx = np.where(np.isin(labels, classes))[0]
    X = data_set[idx]
    y = np.where(labels[idx] == classes[0], -1, 1)
    if random_state is not None:
        rng = np.random.default_rng(random_state)
        shuffle = rng.permutation(len(X))
        X, y = X[shuffle], y[shuffle]
    split = int(np.ceil(train_ratio * len(X)))
    return X[:split], X[split:], y[:split], y[split:]

In [260]:
def get_labels_count(data_set):
    return len(data_set)

Use the code that we have implemented earlier:

In [261]:
def train(train_data_set, train_labels, kernel_type='linear', C=10.0, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)
    objects_count = train_data_set.shape[0]

    P = train_labels @ train_labels.T * kernel
    q = -np.ones((objects_count, 1))
    G = np.vstack((np.eye(objects_count), -np.eye(objects_count)))
    h = np.vstack((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))
    A = train_labels.reshape(1, objects_count).astype(float)
    b_scalar = 0.0

    sol = cvxopt.solvers.qp(
        cvxopt.matrix(P),
        cvxopt.matrix(q),
        cvxopt.matrix(G),
        cvxopt.matrix(h),
        cvxopt.matrix(A),
        cvxopt.matrix(b_scalar)
    )

    alphas_full = np.array(sol['x']).ravel()
    support_vectors_id = np.where(alphas_full > threshold)[0]
    vector_number = support_vectors_id.size
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = alphas_full[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for idx in support_vectors_id:
        K_row = kernel[idx, support_vectors_id].reshape(vector_number, 1)
        b -= np.sum(lambdas * targets * K_row)
    b /= vector_number

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

def build_kernel(data_set, kernel_type='linear'):
    K = data_set @ data_set.T
    if kernel_type == 'rbf':
        sigma = 1.0
        n = data_set.shape[0]
        ones = np.ones((n, 1))
        diag_K = np.diag(K).reshape(-1, 1)
        K -= 0.5 * (diag_K @ ones.T + ones @ diag_K.T)
        K = np.exp(K / (2.0 * sigma**2))
    return K

def classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    sigma = 1.0
    K_test = test_data_set @ support_vectors.T
    x_norm = np.sum(test_data_set**2, axis=1)[:, None]
    z_norm = np.sum(train_data_set @ train_data_set.T, axis=1)[support_vectors_id][None, :]
    D2 = x_norm + z_norm - 2 * K_test
    K_rbf = np.exp(-D2 / (2.0 * sigma**2))
    scores = K_rbf @ (lambdas * targets) + b
    return np.sign(scores)


In [262]:
iris = load_iris()
data_set = iris.data
labels = iris.target

data_set = data_set[labels!=2]
labels = labels[labels!=2]

train_data_set, test_data_set, train_labels, test_labels = choose_set_for_label(data_set, labels, (0,1), train_ratio=0.7, random_state=4)  
objects_count = get_labels_count(train_labels)
    
lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='rbf')
predicted = classify_rbf(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))

print(accuracy_score(predicted, test_labels))

     pcost       dcost       gap    pres   dres
 0:  2.1410e+00 -8.0567e+02  2e+03  2e-01  5e-15
 1:  2.0926e+00 -2.7880e+01  5e+01  3e-03  5e-15
 2:  1.0277e+00 -1.6007e+00  3e+00  4e-05  4e-15
 3:  1.1338e-01 -2.5927e-01  4e-01  2e-16  3e-15
 4: -6.8369e-03 -5.7729e-02  5e-02  2e-16  1e-15
 5: -1.7559e-02 -2.3830e-02  6e-03  2e-16  4e-16
 6: -1.9467e-02 -2.1359e-02  2e-03  2e-16  3e-16
 7: -2.0121e-02 -2.0618e-02  5e-04  2e-16  2e-16
 8: -2.0365e-02 -2.0401e-02  4e-05  2e-16  2e-16
 9: -2.0379e-02 -2.0380e-02  1e-06  2e-16  2e-16
10: -2.0380e-02 -2.0380e-02  1e-08  2e-16  2e-16
Optimal solution found.
0.7
